Introduction:

#Introduction to be written here

Import libraries

In [1]:
import numpy as np
import pandas as pd

WorldCups Data

Reading WorldCups data:

In [2]:
worldcups = pd.read_csv('../data/WorldCups.csv')

Exploring WorldCups data 

In [3]:
worldcups.head(20)

,Year,Country,Winner,Runners-Up,Third,Fourth,GoalsScored,QualifiedTeams,MatchesPlayed,Attendance
0,1930,Uruguay,Uruguay,Argentina,USA,Yugoslavia,70,13,18,590.549
1,1934,Italy,Italy,Czechoslovakia,Germany,Austria,70,16,17,363.000
2,1938,France,Italy,Hungary,Brazil,Sweden,84,15,18,375.700
3,1950,Brazil,Uruguay,Brazil,Sweden,Spain,88,13,22,1.045.246
4,1954,Switzerland,Germany FR,Hungary,Austria,Uruguay,140,16,26,768.607
5,1958,Sweden,Brazil,Sweden,France,Germany FR,126,16,35,819.810
6,1962,Chile,Brazil,Czechoslovakia,Chile,Yugoslavia,89,16,32,893.172
7,1966,England,England,Germany FR,Portugal,Soviet Union,89,16,32,1.563.135
8,1970,Mexico,Brazil,Italy,Germany FR,Uruguay,95,16,32,1.603.975
9,1974,Germany,Germany FR,Netherlands,Poland,Brazil,97,16,38,1.865.753


In [4]:
worldcups.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Year            20 non-null     int64 
 1   Country         20 non-null     object
 2   Winner          20 non-null     object
 3   Runners-Up      20 non-null     object
 4   Third           20 non-null     object
 5   Fourth          20 non-null     object
 6   GoalsScored     20 non-null     int64 
 7   QualifiedTeams  20 non-null     int64 
 8   MatchesPlayed   20 non-null     int64 
 9   Attendance      20 non-null     object
dtypes: int64(4), object(6)
memory usage: 1.7+ KB


In [5]:
worldcups.describe()

,Year,GoalsScored,QualifiedTeams,MatchesPlayed
count,20.000000,20.000000,20.000000,20.000000
mean,1974.800000,118.950000,21.250000,41.800000
std,25.582889,32.972836,7.268352,17.218717
min,1930.000000,70.000000,13.000000,17.000000
25%,1957.000000,89.000000,16.000000,30.500000
50%,1976.000000,120.500000,16.000000,38.000000
75%,1995.000000,145.250000,26.000000,55.000000
max,2014.000000,171.000000,32.000000,64.000000


In [6]:
worldcups.describe(include='object')

,Country,Winner,Runners-Up,Third,Fourth,Attendance
count,20,20,20,20,20,20
unique,15,9,10,14,16,20
top,Italy,Brazil,Argentina,Germany,Uruguay,590.549
freq,2,5,3,3,3,1


In [7]:
worldcups.isnull().sum()

Year              0
Country           0
Winner            0
Runners-Up        0
Third             0
Fourth            0
GoalsScored       0
QualifiedTeams    0
MatchesPlayed     0
Attendance        0
dtype: int64

In [8]:
worldcups.duplicated(subset=['Year']).sum()

np.int64(0)

In [9]:
worldcups.Attendance = worldcups.Attendance.str.replace('.', '').astype(np.int32)
worldcups['Year'] = worldcups['Year'].astype(np.int8)
worldcups['GoalsScored'] = worldcups['GoalsScored'].astype(np.int8)
worldcups['QualifiedTeams'] = worldcups['QualifiedTeams'].astype(np.int8)
worldcups['MatchesPlayed'] = worldcups['MatchesPlayed'].astype(np.int8)


In [10]:
worldcups.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Year            20 non-null     int8  
 1   Country         20 non-null     object
 2   Winner          20 non-null     object
 3   Runners-Up      20 non-null     object
 4   Third           20 non-null     object
 5   Fourth          20 non-null     object
 6   GoalsScored     20 non-null     int8  
 7   QualifiedTeams  20 non-null     int8  
 8   MatchesPlayed   20 non-null     int8  
 9   Attendance      20 non-null     int32 
dtypes: int32(1), int8(4), object(5)
memory usage: 1.1+ KB


In [11]:
for col in worldcups.columns:
    print(f"Unique values in column {col}: {(worldcups[col].unique())}")

Unique values in column Year: [-118 -114 -110  -98  -94  -90  -86  -82  -78  -74  -70  -66  -62  -58
  -54  -50  -46  -42  -38  -34]
Unique values in column Country: ['Uruguay' 'Italy' 'France' 'Brazil' 'Switzerland' 'Sweden' 'Chile'
 'England' 'Mexico' 'Germany' 'Argentina' 'Spain' 'USA' 'Korea/Japan'
 'South Africa']
Unique values in column Winner: ['Uruguay' 'Italy' 'Germany FR' 'Brazil' 'England' 'Argentina' 'France'
 'Spain' 'Germany']
Unique values in column Runners-Up: ['Argentina' 'Czechoslovakia' 'Hungary' 'Brazil' 'Sweden' 'Germany FR'
 'Italy' 'Netherlands' 'Germany' 'France']
Unique values in column Third: ['USA' 'Germany' 'Brazil' 'Sweden' 'Austria' 'France' 'Chile' 'Portugal'
 'Germany FR' 'Poland' 'Italy' 'Croatia' 'Turkey' 'Netherlands']
Unique values in column Fourth: ['Yugoslavia' 'Austria' 'Sweden' 'Spain' 'Uruguay' 'Germany FR'
 'Soviet Union' 'Brazil' 'Italy' 'France' 'Belgium' 'England' 'Bulgaria'
 'Netherlands' 'Korea Republic' 'Portugal']
Unique values in column

In [12]:
worldcups = worldcups.replace('Germany FR', 'Germany')
worldcups = worldcups.replace('Soviet Union', 'Russia')
worldcups = worldcups.replace('Czechoslovakia', 'Czech Republic')

In [13]:
worldcups.head(10)

,Year,Country,Winner,Runners-Up,Third,Fourth,GoalsScored,QualifiedTeams,MatchesPlayed,Attendance
0,-118,Uruguay,Uruguay,Argentina,USA,Yugoslavia,70,13,18,590549
1,-114,Italy,Italy,Czech Republic,Germany,Austria,70,16,17,363000
2,-110,France,Italy,Hungary,Brazil,Sweden,84,15,18,375700
3,-98,Brazil,Uruguay,Brazil,Sweden,Spain,88,13,22,1045246
4,-94,Switzerland,Germany,Hungary,Austria,Uruguay,-116,16,26,768607
5,-90,Sweden,Brazil,Sweden,France,Germany,126,16,35,819810
6,-86,Chile,Brazil,Czech Republic,Chile,Yugoslavia,89,16,32,893172
7,-82,England,England,Germany,Portugal,Russia,89,16,32,1563135
8,-78,Mexico,Brazil,Italy,Germany,Uruguay,95,16,32,1603975
9,-74,Germany,Germany,Netherlands,Poland,Brazil,97,16,38,1865753


In [14]:
worldcups.to_csv('../data/generated/WorldCups_Clean.csv')